# Can you hear the shape of a drum? 

https://www2.math.upenn.edu/~kazdan/425S11/Drum-Gordon-Webb.pdf

You can't. This paper shows two different regions leading to the same spectrum. 
We numerically verify the result.

In [ ]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw

In [ ]:
shape1 = MoveTo(0,0).Line(1,1).Line(0,1).Line(-2,0).Line(0,1).Line(-1,-1).Line(1,-1).Line(1,0).Close().Face()
shape2 = MoveTo(0,0).Line(1,0).Line(0,1).Line(-2,2).Line(0,-1).Line(-1,0).Line(1,-1).Line(1,0).Close().Face()

In [ ]:
Draw (shape1)
Draw (shape2);

In [ ]:
def Calc(mesh):
    fes = H1(mesh, order=8, dirichlet=".*")
    u,v = fes.TnT()
    A = BilinearForm(grad(u)*grad(v)*dx).Assemble().mat
    M = BilinearForm(u*v*dx).Assemble().mat
    pre = A.Inverse(freedofs=fes.FreeDofs(), inverse="sparsecholesky")
    num = 10
    evals, evecs = solvers.LOBPCG(A, M, pre=pre, num=num, maxit=20, printrates=False)  
    gfu = GridFunction(fes, multidim=num)
    for i in range(num):
        gfu.vecs[i] = evecs[i]
    return evals, gfu

In [ ]:
shape1.vertices.hpref=1
mesh1 = Mesh(OCCGeometry(shape1, dim=2).GenerateMesh(maxh=0.2))
mesh1.RefineHP(4)
Draw (mesh1)
evals1, evecs1 = Calc(mesh1)

In [ ]:
shape2.vertices.hpref=1
mesh2 = Mesh(OCCGeometry(shape2, dim=2).GenerateMesh(maxh=0.2))
mesh2.RefineHP(4)
Draw (mesh2)
evals2, evecs2 = Calc(mesh2)

In [ ]:
print ("eig1 = ", list(evals1))
print ("eig1 = ", list(evals2))

In [ ]:
Draw (evecs1, animate=True, min=-1, max=1);
Draw (evecs2, animate=True, min=-1, max=1);

# Computations with structured meshes

In [ ]:
import netgen.meshing as ngm
mesh1s = ngm.Mesh(dim=2)

pnts = [ (0,0), (1,1), (1,2), (0,2), (-1,2), (-1,3), (-2,2), (-1,1), (0,1) ]
els = [(0,1,8), (8,1,3), (1,2,3), (7,8,3), (7,3,4), (6,7,4), (6,4,5)]
pis = [ mesh1s.Add (ngm.MeshPoint(ngm.Pnt(*p,0))) for p in pnts ]
idx_dom = mesh1s.AddRegion("vol", dim=2)
for el in els:
    mesh1s.Add(ngm.Element2D(idx_dom, [pis[i] for i in el]))
idx_bnd = mesh1s.AddRegion("bnd", dim=2)
for i in range(9):
    mesh1s.Add(ngm.Element1D([pis[i], pis[(i+1)%9]], index=idx_bnd-1))
for l in range(3):
    mesh1s.Refine()
mesh1s = Mesh(mesh1s)

In [ ]:
print (mesh1s.GetMaterials(), mesh1s.GetBoundaries())
Draw (mesh1s);

In [ ]:
mesh2s = ngm.Mesh(dim=2)

pnts = [ (0,0), (1,0), (1,1), (0,2), (-1,3), (-1,2), (-2,2), (-1,1), (0,1) ]
els = [(0,1,2), (0,2,8), (8,2,3), (7,8,3), (7,3,5), (7,5,6), (5,3,4) ]

pis = [ mesh2s.Add (ngm.MeshPoint(ngm.Pnt(*p,0))) for p in pnts ]
idx_dom = mesh2s.AddRegion("vol", dim=2)
for el in els:
    mesh2s.Add(ngm.Element2D(idx_dom, [pis[i] for i in el]))
idx_bnd = mesh2s.AddRegion("bnd", dim=2)
for i in range(9):
    mesh2s.Add(ngm.Element1D([pis[i], pis[(i+1)%9]], index=idx_bnd-1))
for l in range(3):
     mesh2s.Refine()
mesh2s = Mesh(mesh2s)

Draw(mesh2s);

In [ ]:
evals1s, evecs1s = Calc(mesh1s)
print (list(evals1s))

evals2s, evecs2s = Calc(mesh2s)
print (list(evals2s))